## Introduction

Connect to SQL Database

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

## Overview of the Data

We start by getting a sense of the data.

In [3]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

- <font color='blue'>name</font> - The name of the country.
- <font color='blue'>area</font> - The total land and sea area of the country.
- <font color='blue'>population</font> - The country's population.
- <font color='blue'>population_growth</font> - The country's population growth as a percentage.
- <font color='blue'>birth_rate</font> - The country's birth rate, or the number of births a year per 1,000 people.
- <font color='blue'>death_rate</font> - The country's death rate, or the number of death a year per 1,000 people.
- <font color='blue'>area_land</font> - The country's land area in square kilometers.
- <font color='blue'>area_water</font> - The country's waterarea in square kilometers.

## Summary Statisitcs

In [4]:
%%sql
SELECT min(population) as min_pop,
       max(population) as max_pop,
       min(population_growth) as min_pop_growth,
       max(population_growth) as max_pop_growth
  FROM facts;

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


We must use subqueries to filter some of this data. This is because you can't have a country with a miniumum population of 0 or a maximum population of 7.2 billion.

## Exploring Outliers

In [5]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MIN(population)
                        FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [6]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


Our first query shows us the country of Antarctica. There is an actual true 0 population in the country due to its climate. Also, the result of 7.2 billion population was for the entire world. We now want to revist our summary statistics without this value.

## Summary Statistics Revisited

Let us find the max population of a country that does not include the data row for 'World'.

In [9]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth 
  FROM facts
 WHERE name <> 'World';

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


We have located the most populous country with a population of about 1.4 billion.

Let's take a look at some more statistics from this dataset.

## Exploring Average Population and Area

Let's explore density. Density depends on the country's area and population. First let us look at averages.

In [10]:
%%sql
SELECT AVG(population) as avg_pop,
       AVG(area) as avg_area
  FROM facts
 WHERE name <> 'World';

Done.


avg_pop,avg_area
32242666.56846473,555093.546184739


The average population for a country is 32 million with an average area of 555 thousand sqaure kilometers.

## Finding Densely Population Countries

To meet this criteria, we need to query the countries that have higher than average values for population size, but lower than average values for area size.

In [13]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name <> 'World')
   AND area < (SELECT AVG(area)
                       FROM facts
                      WHERE name <> 'World')

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


These are countries that meet are criteria of high density. I have only been to the country of Poland on this list. From my experience, Poland has a plethora of populated cities that are very dense, but there is tons of open space and forest that are not developed or haas small farms. However, with retrospect to other countries, I could see how it falls into the more dense category

## Top 5 and Bottom 5 Population Growth Countries

Let's take a look at the Top 5 in terms of Population Growth. These are the countries that will be growing in size over the next decade or so which will have major changes in the geopolitical world.

In [18]:
%%sql
SELECT name, population_growth
  FROM facts
 WHERE name <> 'World'
ORDER BY population_growth DESC
LIMIT 5;

Done.


name,population_growth
South Sudan,4.02
Malawi,3.32
Burundi,3.28
Niger,3.25
Uganda,3.24


Let's take a look at the Bottom 5 in terms of population growth. Let's filter any row with no data.

In [19]:
%%sql
SELECT name, population_growth
  FROM facts
 WHERE population_growth <> 'None'
ORDER BY population_growth
LIMIT 5;

Done.


name,population_growth
Holy See (Vatican City),0.0
Cocos (Keeling) Islands,0.0
Greenland,0.0
Pitcairn Islands,0.0
Greece,0.01


Let's try to see all the countries with lower birthrates that are above 250 square miles.

In [20]:
%%sql
SELECT name, population_growth
  FROM facts
 WHERE population_growth <> 'None' and area > 250
ORDER BY population_growth
LIMIT 5;

Done.


name,population_growth
Greenland,0.0
Greece,0.01
Falkland Islands (Islas Malvinas),0.01
Guyana,0.02
Slovakia,0.02


These countries will need to increase their population growth if they want to compete in the 21st century, and for the benefit of their economies.

## Water Area / Area Ratio

Let's look at one more intresting stat. We will compute the ratio of water square miles by the total area of square miles for every country, and find out the countries with highest amount of land that is water compared to it's whole area.

In [33]:
%%sql
SELECT name, ROUND((CAST(area_water as FLOAT) / CAST(area as FLOAT) * 100),1) as Water_Ratio_Percent
  FROM facts
ORDER BY Water_Ratio_Percent DESC
LIMIT 10;

Done.


name,Water_Ratio_Percent
British Indian Ocean Territory,99.9
Virgin Islands,81.9
Puerto Rico,35.7
"Bahamas, The",27.9
Guinea-Bissau,22.2
Malawi,20.6
Netherlands,18.4
Uganda,18.2
Eritrea,14.1
Liberia,13.5


Not suprisngly, many of the countries with highest percentage of Water Area compared to land area are islands. However, one is suprised to see some Africa's countries in the mix, considering the much of what people assume of Africa as being a rather dry climate!